In [46]:
import requests
import json
import time
from datetime import datetime, timezone
from web3 import Web3
from eth_abi import decode_abi

In [12]:
''' connect to a JSON-RPC node '''
# Replace with your own Alchemy (or Infura) RPC URL
alchemy_url = "https://eth-mainnet.g.alchemy.com/v2/XtWfn0d5ooMWIVs5AQMxOAb474Km-hs7"

w3 = Web3(Web3.HTTPProvider(alchemy_url))

# Check connection
print("Connected:", w3.is_connected())


Connected: True


In [47]:
ALCHEMY_URL = "https://eth-mainnet.g.alchemy.com/v2/XtWfn0d5ooMWIVs5AQMxOAb474Km-hs7"

In [3]:
''' v3 factory '''

UNISWAP_V3_FACTORY = "0x1F98431c8aD98523631AE4a59f267346ea31F984"
ETHERSCAN_API_KEY = "NP263JVHJEZUFCC72UQKX93JTDZJ8WPGUW"

url = f"https://api.etherscan.io/api?module=contract&action=getabi&address={UNISWAP_V3_FACTORY}&apikey={ETHERSCAN_API_KEY}"
abi = json.loads(requests.get(url).json()['result'])

factory = w3.eth.contract(address=UNISWAP_V3_FACTORY, abi=abi)


In [64]:
TOAI = Web3.to_checksum_address("0xEd291c4EA45be02481E7f8359cAaDA27d5909f42")
WETH  = Web3.to_checksum_address("0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2")

# Uniswap V3 fee tiers
FEE_TIERS = [100, 500, 3000, 10000]  # 0.01%, 0.05%, 0.3%, 1%

# def get_existing_pools(tokenA, tokenB):
#     pools = {}
#     for fee in FEE_TIERS:
#         try:
#             pool = factory.functions.getPool(tokenA, tokenB, fee).call()
#             if int(pool, 16) != 0:
#                 pools[fee] = pool
#         except:
#             continue
#     return pools

# # Try both orders: TOAI/WETH and WETH/TOAI
# pools = get_existing_pools(TOAI, WETH)
# if not pools:
#     pools = get_existing_pools(WETH, TOAI)

# print("🧠 Found pools:")
# for fee, addr in pools.items():
#     print(f"Fee tier {fee/100000:.2%}: {addr}")


def get_existing_pools(tokenA, tokenB):
    pools = {}
    token0, token1 = sorted([tokenA, tokenB])
    for fee in FEE_TIERS:
        try:
            pool = factory.functions.getPool(token0, token1, fee).call()
            if int(pool, 16) != 0:
                pools[fee] = pool
        except:
            continue
    return pools

pools = get_existing_pools(TOAI, WETH)


In [21]:
def get_contract_creation_block(address, api_key):
    url = f"https://api.etherscan.io/api?module=contract&action=getcontractcreation&contractaddresses={address}&apikey={api_key}"
    res = requests.get(url).json()
    if res["status"] == "1":
        return int(res["result"][0]["blockNumber"])
    return None

pool_address = "0xEd291c4EA45be02481E7f8359cAaDA27d5909f42"
creation_block = get_contract_creation_block(pool_address, ETHERSCAN_API_KEY)
print("Pool creation block:", creation_block)

timestamp = w3.eth.get_block(creation_block)["timestamp"]
from datetime import datetime
print("Pool created at:", datetime.utcfromtimestamp(timestamp))


Pool creation block: 22426269
Pool created at: 2025-05-06 17:51:11


In [65]:
import requests

from_block = hex(22426269)
to_block = hex(22426269 + 499)
token_address = "0x34E76b60C6fe4572310E29c7DeeFb4f629485837"

# eth_getLogs (POST /:apiKey)
response = requests.post(
  "https://eth-mainnet.g.alchemy.com/v2/XtWfn0d5ooMWIVs5AQMxOAb474Km-hs7",
  headers={},
json={
    "jsonrpc": "2.0",
    "method": "eth_getLogs",
    "params": [
      {
        "fromBlock": from_block,
        "toBlock": to_block,
        "address": token_address
      }
    ],
    "id": 1
  },
)

print(response.json())

{'jsonrpc': '2.0', 'id': 1, 'result': [{'address': '0x34e76b60c6fe4572310e29c7deefb4f629485837', 'topics': ['0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef', '0x0000000000000000000000000000000000000000000000000000000000000000', '0x0000000000000000000000000000000000000000000000000000000000000000'], 'data': '0x00000000000000000000000000000000000000000000000000000000000003e8', 'blockHash': '0xb6f2769fe6bc02bd159aa6bb0c9274a6ddd3b81f1c1b676f8f14353324bc3707', 'blockNumber': '0x15632a4', 'blockTimestamp': '0x681a4c63', 'transactionHash': '0x8c0c1d47186a028dedf8dbf0131fa219e2f25dd9d7bcf0774a0d9187751614fc', 'transactionIndex': '0x7', 'logIndex': '0xd6', 'removed': False}, {'address': '0x34e76b60c6fe4572310e29c7deefb4f629485837', 'topics': ['0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef', '0x0000000000000000000000000000000000000000000000000000000000000000', '0x00000000000000000000000063083c54fb26e757a1833dd1463262acf8854a01'], 'data': '0x00000000000000

In [50]:
# Step-1: Get pool address
# Step-2: Get SWAP event signature
# Step-3: Query SWAP data
# Step-4: Decode SWAP log data
# Step-5: Convert block no. --> timestamp
# step-6: Aggregate into OHLCV 

In [66]:
''' step-2 '''

# Uniswap V3 Swap event signature for decoding logs:
# event Swap(
#     address indexed sender,
#     address indexed recipient,
#     int256 amount0,
#     int256 amount1,
#     uint160 sqrtPriceX96,
#     uint128 liquidity,
#     int24 tick
# )



SWAP_EVENT_TOPIC = Web3.keccak(text="Swap(address,address,int256,int256,uint160,uint128,int24)").hex()
print("V3 Swap Event Topic:", SWAP_EVENT_TOPIC)


V3 Swap Event Topic: 0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67


In [67]:
''' step-3 '''

def get_swap_logs(pool_address, from_block, to_block):
    logs = []

    while from_block < to_block:
        end_block = min(from_block + 499, to_block)
        response = requests.post(
            ALCHEMY_URL,
            json={
                "jsonrpc": "2.0",
                "id": 1,
                "method": "eth_getLogs",
                "params": [{
                    "fromBlock": hex(from_block),
                    "toBlock": hex(end_block),
                    "address": pool_address,
                    "topics": [SWAP_EVENT_TOPIC]
                }]
            }
        )
        result = response.json()
        if "result" in result:
            logs.extend(result["result"])
        from_block = end_block + 1

    return logs


In [69]:
raw_logs = get_swap_logs(pool_address, creation_block, creation_block + 5000)
print(f"✅ Total logs fetched: {len(raw_logs)}")


✅ Total logs fetched: 0


In [45]:
''' step-4 '''

def decode_swap_log(log):
    decoded = decode_abi(
        ['int256', 'int256', 'uint160', 'uint128', 'int24'],
        bytes.fromhex(log['data'][2:])
    )
    amount0, amount1, sqrtPriceX96, liquidity, tick = decoded
    return {
        "amount0": amount0 / (10**18),  # adjust per token decimals
        "amount1": amount1 / (10**18),
        "price": (sqrtPriceX96 / (2**96)) ** 2,
        "tick": tick,
        "block_number": int(log['blockNumber'], 16),
        "tx_hash": log['transactionHash']
    }


In [49]:
''' step-5 '''

timestamp = w3.eth.get_block(block_number)["timestamp"]
datetime.utcfromtimestamp(timestamp).replace(second=0, microsecond=0)


datetime.datetime(2025, 5, 6, 17, 51)

In [52]:
from datetime import datetime

def round_to_minute(ts):
    return datetime.utcfromtimestamp(ts).replace(second=0, microsecond=0)

# ✅ Step 1: Fetch raw logs
raw_logs = get_swap_logs(pool_address, creation_block, creation_block + 1000)

# ✅ Step 2: Decode logs and attach timestamp
decoded_logs = []
for log in raw_logs:
    decoded = decode_swap_log(log)
    block_ts = w3.eth.get_block(decoded['block_number'])['timestamp']
    decoded['timestamp'] = block_ts
    decoded_logs.append(decoded)

# ✅ Step 3: Aggregate into OHLCV
ohlcv = {}
for log in decoded_logs:
    ts_bucket = round_to_minute(log['timestamp'])

    if ts_bucket not in ohlcv:
        ohlcv[ts_bucket] = {
            "open": log['price'],
            "high": log['price'],
            "low": log['price'],
            "close": log['price'],
            "volume": abs(log['amount0']) + abs(log['amount1'])
        }
    else:
        ohlcv[ts_bucket]['high'] = max(ohlcv[ts_bucket]['high'], log['price'])
        ohlcv[ts_bucket]['low'] = min(ohlcv[ts_bucket]['low'], log['price'])
        ohlcv[ts_bucket]['close'] = log['price']
        ohlcv[ts_bucket]['volume'] += abs(log['amount0']) + abs(log['amount1'])

# ✅ Print sample
for ts, data in list(ohlcv.items())[:5]:
    print(ts, data)


In [72]:
import requests
from web3 import Web3
from eth_abi import decode_abi
from datetime import datetime

# Connect to Alchemy
ALCHEMY_URL = "https://eth-mainnet.g.alchemy.com/v2/XtWfn0d5ooMWIVs5AQMxOAb474Km-hs7"
w3 = Web3(Web3.HTTPProvider(ALCHEMY_URL))
print("Connected:", w3.is_connected())

# Swap event topic for Uniswap V3
SWAP_EVENT_TOPIC = "0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67"

# Your pool
pool_address = Web3.to_checksum_address("0x34E76b60C6fe4572310E29c7DeeFb4f629485837")

# Step 1: Fetch swap logs
def get_swap_logs(pool_address, start_block, end_block):
    logs = []
    while start_block <= end_block:
        batch_end = min(start_block + 499, end_block)
        payload = {
            "jsonrpc": "2.0",
            "method": "eth_getLogs",
            "params": [{
                "fromBlock": hex(start_block),
                "toBlock": hex(batch_end),
                "address": pool_address,
                "topics": [SWAP_EVENT_TOPIC]
            }],
            "id": 1
        }
        response = requests.post(ALCHEMY_URL, json=payload)
        result = response.json()
        if "result" in result:
            logs.extend(result["result"])
        start_block = batch_end + 1
    return logs

# Step 2: Decode logs
def decode_swap_log(log):
    decoded = decode_abi(
        ['int256', 'int256', 'uint160', 'uint128', 'int24'],
        bytes.fromhex(log['data'][2:])
    )
    return {
        "amount0": decoded[0] / 1e18,  # adjust decimals if needed
        "amount1": decoded[1] / 1e18,
        "price": (decoded[2] / (2**96))**2,
        "tick": decoded[4],
        "block_number": int(log['blockNumber'], 16),
        "tx_hash": log['transactionHash']
    }

# Step 3: Fetch and decode
start_block = 22426269 
end_block = start_block + 5000

raw_logs = get_swap_logs(pool_address, start_block, end_block)
print(f"✅ Logs fetched: {len(raw_logs)}")

decoded_logs = []
for log in raw_logs:
    data = decode_swap_log(log)
    block_ts = w3.eth.get_block(data['block_number'])['timestamp']
    data['timestamp'] = block_ts
    decoded_logs.append(data)

# Step 4: Print sample
for entry in decoded_logs[:5]:
    print(datetime.utcfromtimestamp(entry["timestamp"]), entry)


Connected: True
✅ Logs fetched: 0


In [74]:
import requests
from web3 import Web3
from eth_abi import decode_abi
from datetime import datetime

# Connect to Alchemy
ALCHEMY_URL = "https://eth-mainnet.g.alchemy.com/v2/XtWfn0d5ooMWIVs5AQMxOAb474Km-hs7"
w3 = Web3(Web3.HTTPProvider(ALCHEMY_URL))
print("Connected:", w3.is_connected())

# ✅ Swap event topic for Uniswap V2
SWAP_TOPIC_V2 = Web3.keccak(
    text="Swap(address,uint256,uint256,uint256,uint256,address)"
).hex()

# ✅ Your V2 pool address
pool_address = Web3.to_checksum_address("0x34E76b60C6fe4572310E29c7DeeFb4f629485837")

# Step 1: Fetch logs (Uniswap V2 - 499 block chunks)
def get_swap_logs(pool_address, start_block, end_block):
    logs = []
    while start_block <= end_block:
        batch_end = min(start_block + 499, end_block)
        payload = {
            "jsonrpc": "2.0",
            "method": "eth_getLogs",
            "params": [{
                "fromBlock": hex(start_block),
                "toBlock": hex(batch_end),
                "address": pool_address,
                "topics": [SWAP_TOPIC_V2]
            }],
            "id": 1
        }
        response = requests.post(ALCHEMY_URL, json=payload)
        result = response.json()
        if "result" in result:
            logs.extend(result["result"])
        start_block = batch_end + 1
    return logs

# Step 2: Decode Uniswap V2 logs
def decode_v2_swap_log(log):
    decoded = decode_abi(
        ['uint256', 'uint256', 'uint256', 'uint256'],
        bytes.fromhex(log['data'][2:])
    )
    return {
        "amount0In": decoded[0] / 1e18,
        "amount1In": decoded[1] / 1e18,
        "amount0Out": decoded[2] / 1e18,
        "amount1Out": decoded[3] / 1e18,
        "block_number": int(log['blockNumber'], 16),
        "tx_hash": log['transactionHash']
    }

# Step 3: Fetch & Decode logs
start_block = 22426269
end_block = start_block + 5000

raw_logs = get_swap_logs(pool_address, start_block, end_block)
print(f"✅ Logs fetched: {len(raw_logs)}")

decoded_logs = []
for log in raw_logs:
    data = decode_v2_swap_log(log)
    block_ts = w3.eth.get_block(data['block_number'])['timestamp']
    data['timestamp'] = block_ts
    decoded_logs.append(data)

# Step 4: Print sample logs
for entry in decoded_logs[:5]:
    print(datetime.utcfromtimestamp(entry["timestamp"]), entry)


Connected: True
✅ Logs fetched: 204
2025-05-06 18:29:59 {'amount0In': 0.015229490645110184, 'amount1In': 0.0, 'amount0Out': 0.0, 'amount1Out': 73154.0, 'block_number': 22426462, 'tx_hash': '0xb333ac7b3f45d7de268900e2218a19fcf0a2664d6936f4767eb3aa2c5a3f4dff', 'timestamp': 1746556199}
2025-05-06 18:29:59 {'amount0In': 0.009450294153242554, 'amount1In': 0.0, 'amount0Out': 0.0, 'amount1Out': 44660.0, 'block_number': 22426462, 'tx_hash': '0x8d9d609017af4338e5123e05d892c2ec7a080ff452a363fff71f0fc5d717a2c3', 'timestamp': 1746556199}
2025-05-06 18:29:59 {'amount0In': 0.01453245250243466, 'amount1In': 0.0, 'amount0Out': 0.0, 'amount1Out': 67609.0, 'block_number': 22426462, 'tx_hash': '0xb0647d02b6d371e6e2115917218d79c7cdfed7c423684b30b22225cae0a64370', 'timestamp': 1746556199}
2025-05-06 18:29:59 {'amount0In': 0.015084707507008448, 'amount1In': 0.0, 'amount0Out': 0.0, 'amount1Out': 68843.0, 'block_number': 22426462, 'tx_hash': '0xec48a4ef77561ed2b92e6ad54022487becf76c43e88d93d8ae945650a2855547'